In [ ]:
pip install plotly

In [ ]:
pip install seaborn

In [ ]:
!git clone https://github.com/miguel-fc/Neutron_Reflect.git

In [ ]:
cd Neutron_Reflect

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import numpy as np
import data_preparation as dpre
import io
import os
# import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline
import seaborn as sea

In [ ]:
pip install refl1d

In [ ]:
pip install gdown

In [ ]:
pip install ipympl

In [ ]:
#function to convert SLD back to NR curve
import refl1d
from refl1d.names import *

def calculate_reflectivity_from_profile(q, z_step, sld, q_resolution=0.0294855):

    """

        Reflectivity calculation using refl1d from an array of microslabs

    """

    # sld = np.flip(sld)
    zeros = np.zeros(len(q))
    dq = q_resolution * q / 2.355
    # The QProbe object represents the beam

    probe = QProbe(q, dq, data=(zeros, zeros))
    sample = Slab(material=SLD(name='back', rho=sld[0], irho=0), interface=0)
    # Add each layer
    _prev_z = z_step[0]
    for i, _sld in enumerate(sld):
        if i>0:
            thickness = z_step[i] - _prev_z
            sample = sample | Slab(material=SLD(name='l_%d' % i, rho=_sld, irho=0),
                                                thickness=thickness,
                                                interface=0)
        _prev_z = z_step[i]
    probe.background = Parameter(value=0, name='background')
    expt = Experiment(probe=probe, sample=sample)
    a, r = expt.reflectivity()
    return a, r

In [ ]:
import gdown
gdown.download_folder("https://drive.google.com/drive/u/0/folders/10zAh8Dqr9c-vSflP35e6KFh54XdYQ-mr")

In [ ]:
#load in experimental NR curves
import os
directory = "npoutputsPoly/"
totalDataPoly = []
parametersPoly = []
count = 0
for i,filename in enumerate(os.listdir(directory)):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(f[14:])
    currData = np.load(directory +f[14:])
    xptPoly = []
    yptPoly = []
    parametersPoly.append([])
    for j in range(10):
        parametersPoly[count].append(int(f[26:32]))
    for j in range(len(currData)):
      xptPoly.append(currData[j][0])
      yptPoly.append(currData[j][1])
    np.array(xptPoly)
    np.array(yptPoly)
    totalDataPoly.append([xptPoly,yptPoly])
    if i == 8:
      print(f)
    count += 1
totalDataPoly = np.stack(totalDataPoly)
for x in range(len(totalDataPoly)):
  plt.plot(totalDataPoly[x][0],totalDataPoly[x][1])
plt.xscale('log')
plt.yscale('log')
plt.show()
print(totalDataPoly.shape)
totalDataPoly = np.log10(totalDataPoly)

In [ ]:
#CNN structure

class CNN(nn.Module):
    def __init__(self, num_layers):
        super(CNN, self).__init__()
        self.layers = nn.ModuleList()
        addition = 255/num_layers
        curr = 1
        for hdim in range(num_layers-1):
            self.layers.append(nn.Conv1d(int(curr+0.5), int(curr+addition+0.5),51,padding=25))
            self.layers.append(nn.BatchNorm1d(int(curr+addition+0.5)))
            self.layers.append(nn.ReLU(True))
            curr += addition
        self.layers.append(nn.Conv1d(int(curr+0.5), 256, 51, padding=25))

        
        self.linear1 = nn.Linear(256*308,900*2)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)

        x = x.reshape(-1, 2, 900)
        return torch.sigmoid(x)

In [ ]:
#experimental NR normalization. normalizing the same way that we normalize training data

with open('../NRRanges100000.npy', 'rb') as f:
    min_valXNR, max_valXNR, min_valYNR, max_valYNR = np.load(f)
x_pointsPoly = []
y_pointsPoly = []
for curve in totalDataPoly:
    x_pointsPoly.append(curve[0])
    y_pointsPoly.append(curve[1])
for i in range(len(y_pointsPoly)):
    for j in range(len(y_pointsPoly[0])):
        x_pointsPoly[i][j] -= min_valXNR
        y_pointsPoly[i][j] -= min_valYNR
        x_pointsPoly[i][j] /= (max_valXNR - min_valXNR)
        y_pointsPoly[i][j] /= (max_valYNR - min_valYNR)
totalDataPoly2 = []
for i in range(len(y_pointsPoly)):
    totalDataPoly2.append([x_pointsPoly[i],y_pointsPoly[i]])
totalDataPoly2 = np.stack(totalDataPoly2)
totalDataPoly2.shape
for x in range(totalDataPoly.shape[0]):
  plt.plot(totalDataPoly[x][0],totalDataPoly[x][1])

In [ ]:
#splitting data
totalDataPoly.shape
RPoly = totalDataPoly[:,1]
parametersPoly = np.stack(parametersPoly)
batch_size=64
RPoly_m = RPoly[:,np.newaxis,:]
xtrainPoly, ytrainPoly, xvalPoly, yvalPoly, xtestPoly, ytestPoly = \
dpre.split_input_arrays(RPoly_m,parametersPoly, size_split=0.9)
#Prepare data files, continuation
train_datasetPoly, valid_datasetPoly, test_datasetPoly, train_loaderPoly, valid_loaderPoly, test_loaderPoly = \
dpre.get_dataloaders_fromsplitarrays(xtrainPoly,ytrainPoly,xvalPoly,yvalPoly,xtestPoly,ytestPoly,batch_size=batch_size)
print(train_datasetPoly)

In [ ]:
#plotting NR, SLD->NR curve, and SLD curve. First two are on same graph. Original experimental NR is points, SLD->NR curve is a line (red)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
experimental = []
SLD_curves = []
for i in range(len(totalDataPoly)):
    experimental.append([])
    SLD_curves.append([])
for first in range(1,6):
    for second in range(1,6):
        #SLD de-normalization value
        with open('../SLDRanges100000.npy', 'rb') as f:
            min_valX, max_valX, min_valY, max_valY = np.load(f)
        Model = CNN(12)
        #loading in CNN model
        Model.load_state_dict(torch.load("../PolyzwitterionModels30000/ModelPoly30000" + str(first) + str(second),map_location='cuda'))
        Model.eval()
        loss_fn = torch.nn.MSELoss()
        count = -1
        for i in range(len(valid_datasetPoly)):
            count += 1
            data, label = valid_datasetPoly[i]
            data = data[:,np.newaxis,:].to(device)
            print(label[0])
            Model.eval().to(device)
            with torch.no_grad():
              decoded_img  = Model(data)
            for j in range(len(decoded_img[0][0])):
                # x_points[i][j] -= min_valX
                decoded_img[0][0][j] *= (max_valX - min_valX)
                decoded_img[0][0][j] += min_valX
                decoded_img[0][1][j] *= (max_valY- min_valY)
                decoded_img[0][1][j] += min_valY
            plt.style.use("seaborn-darkgrid")
            plt.plot(decoded_img.cpu()[0][0], decoded_img.cpu()[0][1], linewidth=4, color="g")
            plt.xlabel("z [Å]")
            plt.ylabel("SLD [$10^{-6}Å^{-2}$]")
            plt.title("Scattering Length Density Profile")
            plt.show()
            #converting SLD back to NR
            a, r = calculate_reflectivity_from_profile(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308), decoded_img.cpu()[0][0], decoded_img.cpu()[0][1])
            #original experimental data
            other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
            yPoints = []
            xPoints = []
            for i in range(len(other)):
                xPoints.append(other[i][0])
                yPoints.append(other[i][1])
            print(len(other))
            plt.plot(a, r, linewidth=2, color='r', label="Reproduced NR curve from SLD profile")

            plt.scatter(xPoints,yPoints, linewidth=0.01, label="Original Experimental NR curve")
            plt.xscale("log")
            plt.xlabel("q [1/Å]")
            plt.ylabel("r [Å]")
            plt.title("Neutron Reflectivity")
            plt.yscale("log")
            plt.legend()
            plt.show()
            experimental[count].append(r)
            SLD_curves[count].append([decoded_img.cpu()[0][0], decoded_img.cpu()[0][1]])


        for i in range(len(train_datasetPoly)):
            count += 1
            data, label = train_datasetPoly[i]
            data = data[:,np.newaxis,:].to(device)
            Model.eval().to(device)
            print(label[0])
            with torch.no_grad():
              decoded_img  = Model(data)
            for j in range(len(decoded_img[0][0])):
                decoded_img[0][0][j] *= (max_valX - min_valX)
                decoded_img[0][0][j] += min_valX
                decoded_img[0][1][j] *= (max_valY- min_valY)
                decoded_img[0][1][j] += min_valY
            plt.style.use("seaborn-darkgrid")
            plt.plot(decoded_img.cpu()[0][0], decoded_img.cpu()[0][1], linewidth=4, color="g")
            plt.xlabel("z [Å]")
            plt.ylabel("SLD [$10^{-6}Å^{-2}$]")
            plt.title("Scattering Length Density Profile")
            plt.show()
            a, r = calculate_reflectivity_from_profile(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308), decoded_img.cpu()[0][0], decoded_img.cpu()[0][1])
            other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
            yPoints = []
            xPoints = []
            for i in range(len(other)):
                xPoints.append(other[i][0])
                yPoints.append(other[i][1])
            print(len(other))
            plt.plot(a, r, linewidth=2, color='r', label="Reproduced NR curve from SLD profile")

            plt.scatter(xPoints,yPoints, linewidth=0.01, label="Original Experimental NR curve")
            plt.xscale("log")
            plt.xlabel("q [1/Å]")
            plt.ylabel("r [Å]")
            plt.title("Neutron Reflectivity")
            plt.yscale("log")
            plt.legend()
            plt.show()
            experimental[count].append(r)
            SLD_curves[count].append([decoded_img.cpu()[0][0], decoded_img.cpu()[0][1]])

        for i in range(len(test_datasetPoly)):
            count += 1
            data, label = test_datasetPoly[i]
            data = data[:,np.newaxis,:].to(device)
            Model.eval().to(device)
            print(label[0])
            with torch.no_grad():
              decoded_img  = Model(data)
            for j in range(len(decoded_img[0][0])):
                decoded_img[0][0][j] *= (max_valX - min_valX)
                decoded_img[0][0][j] += min_valX
                decoded_img[0][1][j] *= (max_valY- min_valY)
                decoded_img[0][1][j] += min_valY
            plt.style.use("seaborn-darkgrid")
            plt.plot(decoded_img.cpu()[0][0], decoded_img.cpu()[0][1], linewidth=4, color="g")
            plt.xlabel("z [Å]")
            plt.ylabel("SLD [$10^{-6}Å^{-2}$]")
            plt.title("Scattering Length Density Profile")
            plt.show()
            a, r = calculate_reflectivity_from_profile(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308), decoded_img.cpu()[0][0], decoded_img.cpu()[0][1])
            other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
            yPoints = []
            xPoints = []
            for i in range(len(other)):
                xPoints.append(other[i][0])
                yPoints.append(other[i][1])
            print(len(other))
            plt.plot(a, r, linewidth=2, color='r', label="Reproduced NR curve from SLD profile")

            plt.scatter(xPoints,yPoints, linewidth=0.01, label="Original Experimental NR curve")
            plt.xscale("log")
            plt.xlabel("q [1/Å]")
            plt.ylabel("r [Å]")
            plt.title("Neutron Reflectivity")
            plt.yscale("log")
            plt.legend()
            plt.show()
            experimental[count].append(r)
            SLD_curves[count].append([decoded_img.cpu()[0][0], decoded_img.cpu()[0][1]])



In [ ]:
#NRMSE function

from sklearn.metrics import mean_squared_error 

def error(manual, CNNVersion):
    MSE = mean_squared_error(manual, CNNVersion)
    average = 0
    for i in range(len(manual)):
        average += (manual[i])**2
    # average /= len(manual)
    # MSE /= average**2
    MSE *= len(manual)
    MSE/=average
    return MSE

In [ ]:
#plotting each experimental case such that the results for each experimental case is consecutive
#for example, file 202314's results will be plotted together, and so on.

count = -1
experimentalR2 = []
#This list will contain the following: [NR curve, r^2 value, machine learning model number, SLD curve]
from sklearn.metrics import r2_score
for i in range(len(valid_datasetPoly)):
    count += 1
    data, label = valid_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    yPoints = []
    xPoints = []
    print(label[0])
    min_curr = -float('inf')
    experimentalR2.append([])
    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        yPointsNorm = np.log10(yPoints)
        normOut = np.log10(experimental[count][j])
        min_curr = max(min_curr, abs(error(yPointsNorm, normOut)))
        print(min_curr)
        experimentalR2[count].append([experimental[count][j], error(yPointsNorm, normOut), (first+1, second+1), SLD_curves[count][j]])
        plt.scatter(xPoints, yPoints, linewidth=0.01, label="Original Experimental NR curve")
        plt.plot(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimental[count][j], linewidth=2, color='c', label=str(first)+str(second))
        plt.xlabel("q [1/Å]")
        plt.ylabel("r [Å]")
        plt.title("Neutron Reflectivity")
        plt.yscale("log")
        plt.xscale('log')
        plt.legend()
        plt.show()

for i in range(len(train_datasetPoly)):
    count += 1
    data, label = train_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    print(label[0])

    yPoints = []
    xPoints = []
    min_curr = -float('inf')
    experimentalR2.append([])
    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        yPointsNorm = np.log10(yPoints)
        normOut = np.log10(experimental[count][j])
        min_curr = max(min_curr, abs(error(yPointsNorm, normOut)))
        print(min_curr)
        experimentalR2[count].append([experimental[count][j], error(yPointsNorm, normOut), (first+1, second+1), SLD_curves[count][j]])
        plt.scatter(xPoints, yPoints, linewidth=0.01, label="Original Experimental NR curve")
        plt.plot(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimental[count][j], linewidth=2, color='c', label=str(first)+str(second))
        plt.xlabel("q [1/Å]")
        plt.ylabel("r [Å]")
        plt.title("Neutron Reflectivity")
        plt.yscale("log")
        plt.xscale('log')
        plt.legend()
        plt.show()

for i in range(len(test_datasetPoly)):
    count += 1
    data, label = test_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    yPoints = []
    xPoints = []
    print(label[0])
    min_curr = -float('inf')
    experimentalR2.append([])
    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        yPointsNorm = np.log10(yPoints)
        normOut = np.log10(experimental[count][j])
        min_curr = max(min_curr, abs(error(yPointsNorm, normOut)))
        print(min_curr)
        experimentalR2[count].append([experimental[count][j], error(yPointsNorm, normOut), (first+1, second+1), SLD_curves[count][j]])
        plt.scatter(xPoints, yPoints, linewidth=0.01, label="Original Experimental NR curve")
        plt.plot(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimental[count][j], linewidth=2, color='c', label=str(first)+str(second))
        plt.xlabel("q [1/Å]")
        plt.ylabel("r [Å]")
        plt.title("Neutron Reflectivity")
        plt.yscale("log")
        plt.xscale('log')
        plt.legend()
        plt.show()

In [ ]:
#Same thing as previous block of code but results are sorted in decreasing error measurements

count = -1
from sklearn.metrics import r2_score
for i in range(len(valid_datasetPoly)):
    count += 1
    data, label = valid_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    yPoints = []
    xPoints = []
    print(label[0])
    min_curr = -float('inf')
    experimentalR2[count].sort(key = lambda x: x[1])
    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimentalR2[count])):
        second = int((j)%5)
        first = int((j-(second))/5)
        plt.scatter(xPoints, yPoints, linewidth=0.01, label="Original Experimental NR curve")
        print(experimentalR2[count][j][1])
        plt.plot(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimentalR2[count][j][0], linewidth=2, color='c', label=str(experimentalR2[count][j][2][0])+str(experimentalR2[count][j][2][1]))
        plt.xlabel("q [1/Å]")
        plt.ylabel("r [Å]")
        plt.title("Neutron Reflectivity")
        plt.yscale("log")
        plt.xscale('log')
        plt.legend()
        plt.show()

for i in range(len(train_datasetPoly)):
    count += 1
    data, label = train_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    yPoints = []
    xPoints = []
    print(label[0])
    min_curr = -float('inf')
    experimentalR2[count].sort(key = lambda x: x[1])

    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        print(experimentalR2[count][j][1])
        plt.scatter(xPoints, yPoints, linewidth=0.01, label="Original Experimental NR curve")
        plt.plot(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimentalR2[count][j][0], linewidth=2, color='c', label=str(experimentalR2[count][j][2][0])+str(experimentalR2[count][j][2][1]))
        plt.xlabel("q [1/Å]")
        plt.ylabel("r [Å]")
        plt.title("Neutron Reflectivity")
        plt.yscale("log")
        plt.xscale('log')
        plt.legend()
        plt.show()

for i in range(len(test_datasetPoly)):
    count += 1
    data, label = test_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    yPoints = []
    xPoints = []
    print(label[0])
    min_curr = -float('inf')
    experimentalR2[count].sort(key = lambda x: x[1])

    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        print(experimentalR2[count][j][1])
        plt.scatter(xPoints, yPoints, linewidth=0.01, label="Original Experimental NR curve")
        plt.plot(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimentalR2[count][j][0], linewidth=2, color='c', label=str(experimentalR2[count][j][2][0])+str(experimentalR2[count][j][2][1]))
        plt.xlabel("q [1/Å]")
        plt.ylabel("r [Å]")
        plt.title("Neutron Reflectivity")
        plt.yscale("log")
        plt.xscale('log')
        plt.legend()
        plt.show()


In [ ]:
#function that plots and saves files. Used in next block of code
def plotter(label, SLD, experimental):
    plt.plot(SLD[0], SLD[1], linewidth=4, color="g")
    plt.xlabel("z [Å]")
    plt.ylabel("SLD [$10^{-6}Å^{-2}$]")
    plt.title("Scattering Length Density Profile")
    plt.savefig(str(int(label[0].item()))+ ' SLD Graph.jpeg')
    plt.show()
    plt.scatter(xPoints, yPoints, linewidth=0.01, label="Original Experimental NR curve")
    plt.plot(np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimental, linewidth=2, color='r', label=str(first)+str(second))
    plt.xlabel("q [1/Å]")
    plt.ylabel("r [Å]")
    plt.title("Neutron Reflectivity")
    plt.yscale("log")
    plt.xscale('log')
    plt.legend()
    plt.savefig(str(int(label[0].item())) + ' Neutron Reflectivity Graph.jpeg')
    plt.show()
    np.save(str(int(label[0].item()))+ ' SLD Graph.npy', [SLD[0].numpy(), SLD[1].numpy()])
    np.save(str(int(label[0].item())) + ' Neutron Reflectivity Graph.npy', [np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308),experimental])

In [ ]:
#this is where you can plot then save your best hand-picked results from the graphs above.

count = -1
import math
for i in range(len(valid_datasetPoly)):
    count += 1
    data, label = valid_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    yPoints = []
    xPoints = []
    print(int(label[0].item()), "hello")
    
    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        first += 1
        second += 1
        
        # print(int(label[0].item()), first, second)
        if int(label[0].item()) == 202382 and first == 5 and second == 3:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202300 and first == 5 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202394 and first == 2 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202293 and first == 1 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202413 and first == 4 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202349 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202369 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202314 and first == 3 and second == 3:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202286 and first == 5 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202468 and first == 4 and second == 5:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202279 and first == 2 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202461 and first == 2 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202272 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202437 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202454 and first == 1 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202356 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202425 and first == 4 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202307 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202253 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202265 and first == 3 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202342 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202406 and first == 4 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])


for i in range(len(train_datasetPoly)):
    count += 1
    data, label = train_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    print(label[0])

    yPoints = []
    xPoints = []
    for i in range(len(other)):
        xPoints.append(other[i][0])
        yPoints.append(other[i][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        first += 1
        second += 1
        if int(label[0].item()) == 202382 and first == 5 and second == 3:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202300 and first == 5 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202394 and first == 2 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202293 and first == 1 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202413 and first == 4 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202349 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202369 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202314 and first == 3 and second == 3:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202286 and first == 5 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202468 and first == 4 and second == 5:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202279 and first == 2 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202461 and first == 2 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202272 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202437 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202454 and first == 1 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202356 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202425 and first == 4 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202307 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202253 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202265 and first == 3 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202342 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202406 and first == 4 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])

            
for i in range(len(test_datasetPoly)):
    count += 1
    data, label = test_datasetPoly[i]
    other = np.load("../Neutron_Reflect/npoutputsPoly/np_out_REFL_"+str(int(label[0].item()))+"_combined_data_auto.npy")
    yPoints = []
    xPoints = []
    print(int(label[0].item()), "hello")
    for j in range(len(other)):
        xPoints.append(other[j][0])
        yPoints.append(other[j][1])
    for j in range(len(experimental[0])):
        second = int((j)%5)
        first = int((j-(second))/5)
        first += 1
        second += 1
        print(first, second)
        # print(int(label[0].item()), first, second)
        if int(label[0].item()) == 202382 and first == 5 and second == 3:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202300 and first == 5 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202394 and first == 2 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202293 and first == 1 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202413 and first == 4 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202349 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202369 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202314 and first == 3 and second == 3:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202286 and first == 5 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202468 and first == 4 and second == 5:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202279 and first == 2 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202461 and first == 2 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202272 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202437 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202454 and first == 1 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202356 and first == 3 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202425 and first == 4 and second == 1:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202307 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202253 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202265 and first == 3 and second == 4:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202342 and first == 3 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])
        if int(label[0].item()) == 202406 and first == 4 and second == 2:
            plotter(label, SLD_curves[count][j], experimental[count][j])